In [1]:
from urllib import robotparser
from urllib import request
from urllib.parse import urlparse, urljoin, urlencode, quote_plus, quote, unquote
from urllib.error import HTTPError
import requests
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
from bs4 import BeautifulSoup
from urllib.robotparser import RobotFileParser

In [2]:
robot = robotparser.RobotFileParser()

In [3]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent,urlparse(url)[2])

In [4]:
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
    try:
        resp = requests.request(method, url,
                               params = params if method == 'GET' else {},
                               data = params if method == 'GET' else {},
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit >0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(100)
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}]'.format(e.response.status_code)+url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp
        

In [5]:
resp = download('http://pythonscraping.com/pages/page3.html')
dom = BeautifulSoup(resp.content,'lxml')

In [6]:
node = dom.find('div', {'id':'footer'})

In [10]:
for _ in node.find_parents():
    print(_.name)

div
body
html
[document]


In [13]:
node.find_parents('div')

[<div id="wrapper">
 <img src="../img/gifts/logo.jpg" style="float:left;"/>
 <h1>Totally Normal Gifts</h1>
 <div id="content">Here is a collection of totally normal, totally reasonable gifts that your friends are sure to love! Our collection is
 hand-curated by well-paid, free-range Tibetan monks.<p>
 We haven't figured out how to make online shopping carts yet, but you can send us a check to:<br/>
 123 Main St.<br/>
 Abuja, Nigeria
 We will then send your totally amazing gift, pronto! Please include an extra $5.00 for gift wrapping.</p></div>
 <table id="giftList">
 <tr><th>
 Item Title
 </th><th>
 Description
 </th><th>
 Cost
 </th><th>
 Image
 </th></tr>
 <tr class="gift" id="gift1"><td>
 Vegetable Basket
 </td><td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td><td>
 $15.00
 </td><td>
 <img src="../img/gifts/img1.jpg"/>
 </td></tr>
 <tr class="gift" id="gift

In [14]:
urljoin(resp.request.url, node.find_previous_siblings()[-1]['src'])

'https://www.google.com/img/gifts/logo.jpg'

In [19]:
url = 'https://www.google.com/search'
params = {
    'q': '',
    'oq':'',
    'aqs': 'chrome.0.69i59l4j69i61l2j69i60.1539j1j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = quote('파이썬')
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}


In [22]:
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

[Error] https://www.google.com/search
[400]https://www.google.com/search
400
Bad Request
{'Content-Type': 'text/html; charset=UTF-8', 'Referrer-Policy': 'no-referrer', 'Content-Length': '1555', 'Date': 'Sat, 18 Jul 2020 08:05:23 GMT'}


In [23]:
[(_.text.strip(), _.find_parents('a')[0]['href'])
for _ in dom.find_all('h3',{'class':'LC20lb'})]

[]

In [24]:
url = 'http://minwon.moel.go.kr/minwon2008/lc_minwon/lc_form_apply.do'
params ={
    'cd_appeal_form': 'ALL',
    'srch_gubn': '',
    'srch_cd_appeal_form':'', 
    'search_capp_biz_cd': '',
    'cate_name': '(unable to decode value)',
    'sub_cate_name': '',
    'view_cnt': 10,
    'company_yn': 0,
    'q_string': '',
    'pageIndex': 1,
    'pageSize': 10,
    'pageUnit': 10,
    'large_category_idx': 0,
    'middle_category_idx': 0,
    'tot_middle_category_idx': 1,
    'large_category_cd': '',
    'middle_category_cd': ''
}
headers ={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

In [28]:
resp = download(url, params ,headers, 'POST')
dom = BeautifulSoup(resp.text, 'lxml')

In [32]:
node = dom.find('div', {'class':'cm_table2'})

In [33]:
for _ in node.find_parents():
    print(_.name)

div
form
div
div
div
div
body
html
[document]


In [34]:
node.find_parents('div')[0].find_all(recursive=False)[0]

<div class="location">
<ul>
<li class="home"><a href="/minwon2008/index_new.do">홈</a></li>
<li><span><a href="/minwon2008/lc_minwon/lc_form_apply.do">민원신청</a></span></li>
<li class="spot"><span><strong>서식민원</strong></span></li>
</ul>
</div>

In [35]:
for _ in node.find_previous_siblings():
    print(_.name)

div
div
div
div
h3
div


In [55]:
node.find_previous_siblings()

[<div class="cm_table2_top v2">
 <p class="table_top_noti"> 서식파일은 방문·우편·팩스를 통한 접수에 사용하시기 바랍니다.</p>
 <ul>
 <li><span class="cm_txt_box_blue">신청</span> 민원마당 신청</li>
 <li><span class="cm_txt_box_yellow">이동</span> 연계사이트 이동</li>
 </ul>
 </div>, <div class="table_top">
 <div class="f1">
 <select class="cm_select" id="view_cnt" name="view_cnt" style="width:130px;" title="게시판 목록보기 갯수선택"> <!-- onchange="chgViewCount(this.value);" -->
 <option selected=" selected" value="10">10개씩 보기</option>
 <option value="20">20개씩 보기</option>
 <option value="50">50개씩 보기</option>
 <option value="100">100개씩 보기</option>
 </select>
 <select class="cm_select" id="company_yn" name="company_yn" title="신청대상 선택"> <!-- onchange="srchMinwonDoc('chgTarget');"-->
 <option value="0">전체</option>
 <option value="1">개인</option>
 <option value="2">기업</option>
 </select>
 <span>
 <input class="cm_inptxt" id="q_string" name="q_string" onkeypress="if(window.event.keyCode == 13) srchMinwonDoc();" style="width:300px;ime-mode:active;

In [41]:
from selenium import webdriver

In [43]:
driver = webdriver.Chrome('C:/Users/주상원/Desktop/데이터 캠퍼스/chromedriver')

In [45]:
driver.get(url)
# url 접근

In [54]:
node = dom.find('div', {'class':'cm_table2'})
[td.text.strip() for td in node.find_all('a')]

['임금체불 진정서',
 '',
 '신청',
 '국가기술자격 부정행위 신고서',
 '',
 '신청',
 '산업재해 조사표',
 '',
 '신청',
 '산재발생 미보고 신고서',
 '',
 '신청',
 '직장내 성희롱 신고서',
 '',
 '신청',
 '이주비 청구서',
 '',
 '이동',
 '체당금 등 확인신청서',
 '',
 '신청',
 '감시적,단속적 근로종사자에 대한 적용제외 승인신청서',
 '',
 '신청',
 '체불 임금 등 · 사업주 확인서 발급신청서',
 '',
 '신청',
 '고용보험 피보험자격확인 청구서',
 '',
 '이동']

In [57]:
node.find_all('a', {'class':'over'})

[<a class="over" href="javascript:jsSamuDetail('TMP00000204')">임금체불 진정서</a>,
 <a class="over" href="javascript:jsSamuDetail('TMP00000121')">국가기술자격 부정행위 신고서</a>,
 <a class="over" href="javascript:jsSamuDetail('T9999999999')">산업재해 조사표</a>,
 <a class="over" href="javascript:jsSamuDetail('T9999999999')">산재발생 미보고 신고서</a>,
 <a class="over" href="javascript:jsSamuDetail('T4920000029')">직장내 성희롱 신고서</a>,
 <a class="over" href="javascript:jsSamuDetail('14900000078')">이주비 청구서</a>,
 <a class="over" href="javascript:jsSamuDetail('14900000151')">체당금 등 확인신청서</a>,
 <a class="over" href="javascript:jsSamuDetail('14900000126')">감시적,단속적 근로종사자에 대한 적용제외 승인신청서</a>,
 <a class="over" href="javascript:jsSamuDetail('14900000552')">체불 임금 등 · 사업주 확인서 발급신청서</a>,
 <a class="over" href="javascript:jsSamuDetail('14900000038')">고용보험 피보험자격확인 청구서</a>]

In [59]:
node.find_all('a', {'class':''})

[<a href="javascript:openDownload('/app/minwon_admin/samu/','000007085030_132-0_임금체불진정신고서_작성예제.hwp','');"><img alt="파일(HWP)" src="/img/icn/hwp_icn.gif"/></a>,
 <a href="javascript:openDownload('/app/minwon_admin/samu/','000006863554_부정행위신고서[1].hwp','');"><img alt="파일(HWP)" src="/img/icn/hwp_icn.gif"/></a>,
 <a href="javascript:openDownload('/app/minwon_admin/samu/','000001728501_[별지 제30호서식] 산업재해조사표.hwp','');"><img alt="파일(HWP)" src="/img/icn/hwp_icn.gif"/></a>,
 <a href="javascript:openDownload('/app/minwon_admin/samu/','000000392246_1.산재 미보고 신고서.hwp','');"><img alt="파일(HWP)" src="/img/icn/hwp_icn.gif"/></a>,
 <a href="javascript:openDownload('/app/minwon_admin/samu/','000004919128_1.직장내성희롱 신고서.hwp','');"><img alt="파일(HWP)" src="/img/icn/hwp_icn.gif"/></a>,
 <a href="javascript:openDownload('/app/minwon_admin/samu/','000006618624_[별지 제99호서식] 이주비 청구서.hwp','');"><img alt="파일(HWP)" src="/img/icn/hwp_icn.gif"/></a>,
 <a href="javascript:openDownload('/app/minwon_admin/samu/','000006927243_

In [61]:
resp.request.url

'http://minwon.moel.go.kr/minwon2008/lc_minwon/lc_form_apply.do'

In [79]:
node.find_all('a', {'class':''})

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [88]:
node.find_all('a',{'class':''})[0]['href']

"javascript:openDownload('/app/minwon_admin/samu/','000007085030_132-0_임금체불진정신고서_작성예제.hwp','');"

In [115]:
"_".join(node.find_all('a',{'class':''})[0]['href'].split(",")[1].split('_')[0:4])

"'000007085030_132-0_임금체불진정신고서_작성예제.hwp'"

In [ ]:
#http://minwon.moel.go.kr/comm_jsp/file_Download.jsp?
#attach_path=/app/minwon_admin/samu/&
#attach_title=000007085030_132-0_%C0%D3%B1%DD%C3%BC%BA%D2%C1%F8%C1%A4%BD%C5%B0%ED%BC%AD_%C0%DB%BC%BA%BF%B9%C1%A6.hwp

In [121]:
node.find_all('a',{'class':''})[0]['href'].split(",")[1].split("'")[1]

'000007085030_132-0_임금체불진정신고서_작성예제.hwp'

In [122]:
quote(node.find_all('a',{'class':''})[0]['href'].split(",")[1].split("'")[1])

'000007085030_132-0_%EC%9E%84%EA%B8%88%EC%B2%B4%EB%B6%88%EC%A7%84%EC%A0%95%EC%8B%A0%EA%B3%A0%EC%84%9C_%EC%9E%91%EC%84%B1%EC%98%88%EC%A0%9C.hwp'

In [127]:
node.find_all('a',{'class':''})[0]['href'].split(",")[0].split("'")[1]

'/app/minwon_admin/samu/'

In [129]:
downurl = 'http://minwon.moel.go.kr/comm_jsp/file_Download.jsp?'
attach_path = node.find_all('a',{'class':''})[0]['href'].split(",")[0].split("'")[1]
attach_title = quote(node.find_all('a',{'class':''})[0]['href'].split(",")[1].split("'")[1])

urljoin(downurl, attach_path)+attach_title

'http://minwon.moel.go.kr/app/minwon_admin/samu/000007085030_132-0_%EC%9E%84%EA%B8%88%EC%B2%B4%EB%B6%88%EC%A7%84%EC%A0%95%EC%8B%A0%EA%B3%A0%EC%84%9C_%EC%9E%91%EC%84%B1%EC%98%88%EC%A0%9C.hwp'

In [140]:
def downloadfile(url, params={}, headers={}, method='GET'):
    if canfetch(url) == False:
        print('[Error] ' + url)
    try:
        resp = requests.request(method, url,
                               params = params if method == 'GET' else {},
                               data = params if method == 'GET' else {},
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if e.response.status_code >= 500:
            time.sleep(100)
            resp = download(url, params, headers, method)
        else:
            print('[{}]'.format(e.response.status_code)+url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp
        

In [141]:
downurl = 'http://minwon.moel.go.kr/comm_jsp/file_Download.jsp?'
params = {
    'attach_path' : node.find_all('a',{'class':''})[0]['href'].split(",")[0].split("'")[1],
    'attach_title' : quote(node.find_all('a',{'class':''})[0]['href'].split(",")[1].split("'")[1])
}
resp = downloadfile(downurl, params, headers, 'GET')

KeyboardInterrupt: 

('%EC%9E%91%EC%84%B1%EC%98%88%EC%A0%9C.hwp',
 '%EC%9E%91%EC%84%B1%EC%98%88%EC%A0%9C')

In [95]:
node.find_all('a',{'class':''})[0]['href'].split("/")[4]

"','000007085030_132-0_임금체불진정신고서_작성예제.hwp','');"